### RQ3: Plot the temporal distribution of first and last 10,000 commit messages of random 1000 projects with at least 50,000 commits. Also, perform distribution fitting to finding the best distribution that fits the underlying commit messages at each repository level.

In [3]:
import pandas as pd
import chardet
import nltk
from nltk.corpus import stopwords
import pickle
import pyarrow.parquet as pq
import numpy
import os
import vaex as vx
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as st
import math
import glob
from scipy.optimize import curve_fit
import time
from random import sample
from platform import python_version

#### Using VAEX dataframes for larger 

In [4]:
# VAEX DATAFRAME
df= []

In [5]:
# READ ALL THE PARQUET FILE
df = vx.open('/data/fdac21/eawoc//processed_data/*.parquet')

In [6]:
# COLUMN NAMES
columns = ["projectID", "commitID", "author", "committer", 
           "authorTime", "committerTime", "authorTimezone", 
           "committerTimezone", "commitMessage"]

In [7]:
# Review the data
df.head()

#,projectID,commitID,author,committer,authorTime,committerTime,authorTimezone,committerTimezone,commitMessage
0,wildfly_wildfly,00001a1a9d1552584eee5e62ab11d7b86408c0d8,ochaloup <ochaloup@redhat.com>,ochaloup <ochaloup@redhat.com>,1488898509,1489149067,+0100,+0100,'[JBTM-2858] adding JAXRS filter to import trans...
1,wildfly_wildfly,00010938ae7227834e2c69a9c3c71c95d3f3483d,Stuart Douglas <stuart.w.douglas@gmail.com>,Stuart Douglas <stuart.w.douglas@gmail.com>,1458105079,1458105079,+1100,+1100,WFCORE-1436 Undertow 1.3.19.Final
2,wildfly_wildfly,000183c08f9e7d9770ad8216acb565552da7bfb0,Jeff Mesnil <jmesnil@gmail.com>,Jeff Mesnil <jmesnil@gmail.com>,1332344580,1332436142,+0100,+0100,'[JBPAPP-8184] Connection factory failure\\n* add...
3,wildfly_wildfly,0001a3f7a195b75e563280609636faa249877036,jaikiran <jaikiran.pai@gmail.com>,baranowb <baranowb@gmail.com>,1328294482,1328611298,+0530,+0100,Upgrade to 2.0.0 of jboss-ejb3-ext-api
4,wildfly_wildfly,00024ccfa6fa6fd6426de7bb1d8d494798b384fe,James R. Perkins <jperkins@redhat.com>,GitHub <noreply@github.com>,1583899415,1583899415,-0700,-0700,'Merge pull request #13102 from kabir/WFLY-13222...
5,wildfly_wildfly,0002dbdc7b6cb98e6387cc5f0d11bb93229c5381,Kabir Khan <kkhan@redhat.com>,GitHub <noreply@github.com>,1491037874,1491037874,+0100,+0100,'Merge pull request #9883 from jbosstm/WFLY-8485...
6,wildfly_wildfly,000325d9bb03ceffbe9e7b4756a5566b273040d1,Brian Stansberry <brian.stansberry@redhat.com>,Brian Stansberry <brian.stansberry@redhat.com>,1503172204,1505862581,-0500,-0500,Minor cleanup
7,wildfly_wildfly,0003c4792f52061661cc9db421c7741bc4df8284,Richard Achmatowicz <rachmatowicz@jboss.com>,Paul Ferraro <paul.ferraro@redhat.com>,1370975878,1374591017,-0400,-0400,"[WFLY-1430] Add in XML processing, test cases."
8,wildfly_wildfly,0004ba7ca70b15a48fe6460ec5375c86cd8dc19d,Scott Marlow <smarlow@redhat.com>,Scott Marlow <smarlow@redhat.com>,1463689533,1473273893,-0400,-0400,'add org.wildfly.extension.nosql.common dependen...
9,wildfly_wildfly,000703aae4a2cc2cc0f1c4a9a03ae391bb76def0,Pelisse Romain <belaran@redhat.com>,GitHub <noreply@github.com>,1470623929,1470623929,+0000,+0000,'Merge 03a1e02b4aa392ec3d8914846eee615232f45d89 ...


In [99]:
# list of project names 
project_names = df.projectID.unique()
print(project_names)

['saqib-ali_hivemined', 'git.kernel.org_pub_scm/public-inbox/vger.kernel.org/linux-media/0', 'RIOT-OS_RIOT', 'trattkantarell_src', 'sahil28297_erpnext', 'ThePiGuy0_android_device_motorola_msm8226-common', 'git.kernel.org_pub_scm/public-inbox/vger.kernel.org/lkml/0', 'jdzs1_b', 'ZIDAN44_linux-stable', 'Leaflet_Leaflet', 'timhuynh94_231AS', 'gitorious.org_netbeans/main-silver', 'discourse_discourse', 'nanoframework_ChibiOS-bkp', 'AnnaLut_abs3', 'civicrm_civicrm-core', 'thomasNDS_lbl', 'gentoo_gentoo', 'neo4j_neo4j', 'maritimerabroad_gitfiti', 'dotnet_aspnetcore', 'geos4s_18S856162', 'learn-co-curriculum_python-for-loops-lab', 'jthub_ega-file-transfer-to-collab-1-jtracker', 'denis-gz_qtbase', 'Alsan_turing-chunk07', 'CRAZYFAKE_handiwork', 'i17c_ecode', 'Reese-D_my_emacs', 'gitlab.com_Ponce_slackbuilds', 'jthub_ega-file-transfer-to-collab-2-jtracker', 'stoupoun_p2-breathe', 'Waffiliate_repoForSynch', 'jthub_ega-file-transfer-to-collab-5-jtracker', 'Arshiamidos_84ck1N617', 'mono_mono', 'app

### [3] Plot the temporal distribution of first and last 10,000 commit messages of random 1000 projects with at least 50,000 commits. Also, perform distribution fitting to finding the best distribution that fits the underlying commit messages at each repository level. 

In [2]:
# Use the df data frame for the analysis from above. The commitMessage column contains all the 
# text of the projects. Recommended to work on a smaller sample of commitMessage first. 
# 1. from df select a random project and extract committerTime, sort in the increasing order. 
# Take the first 10k and the last 10k message for plotting. 

### [5.3] Develop static and interactive visualizations to demonstrate the findings.